In [1]:
!pip install tfds-nightly
!pip install tensorflow-hub

     |████████████████████████████████| 3.7MB 9.0MB/s 


In [2]:
import os
import numpy as np
import time
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load compressed models from tensorflow_hub
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


In [3]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCMN2HI/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCMN2HI/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCMN2HI/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(15000)))
y=''
# print(train_data.)
for x in train_examples_batch:
  # y= x.numpy()
  # y=y+' '
  # print(x)
  y=y+x.numpy().decode("utf-8")
  y=y+'\n'
print('Length of text: {} characters'.format(len(y)))

Length of text: 19859096 characters


In [5]:
# The unique characters in the file
vocab = sorted(set(y))
print('{} unique characters'.format(len(vocab)))

169 unique characters


In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in y])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\x08':   0,
  '\t':   1,
  '\n':   2,
  '\x10':   3,
  ' ' :   4,
  '!' :   5,
  '"' :   6,
  '#' :   7,
  '$' :   8,
  '%' :   9,
  '&' :  10,
  "'" :  11,
  '(' :  12,
  ')' :  13,
  '*' :  14,
  '+' :  15,
  ',' :  16,
  '-' :  17,
  '.' :  18,
  '/' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(y[:13]), text_as_int[:13]))

'This was an a' ---- characters mapped to int ---- > [56 76 77 87  4 91 69 87  4 69 82  4 69]


In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(y)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

T
h
i
s
 


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. B"
'oth are great actors, but this must simply be their worst role in history. Even their great acting co'
"uld not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece"
'. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutio'
'ns. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pa'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. "
Target data: "his was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. B"


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 56 ('T')
  expected output: 76 ('h')
Step    1
  input: 76 ('h')
  expected output: 77 ('i')
Step    2
  input: 77 ('i')
  expected output: 87 ('s')
Step    3
  input: 87 ('s')
  expected output: 4 (' ')
Step    4
  input: 4 (' ')
  expected output: 91 ('w')


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 169) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           43264     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 169)           173225    
Total params: 4,154,793
Trainable params: 4,154,793
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([ 58,  92,  42, 153, 112, 138, 136,  42, 101,  56,  21,  96,  78,
       144,  91,  28,  78,  85, 116,  87,  70, 147, 166, 144,  52,   5,
       145, 121, 106,  44,  69, 108,  44,  73,  91,  78,  27,  64,  86,
        42, 102,  31, 141,  25, 103,  60, 126,  99,  68,  58, 100,  53,
        58,   3,  82,  77, 151, 137,  11,  72,  70, 128,  48, 108, 120,
        20,  26,   9, 146,  16, 163,  41, 100, 129, 109, 161,  79, 137,
       135, 158,  91,  91,  97,  19,  89,  57, 147,  40, 117, 140,  82,
       101,   5, 102, 157,  57,  64, 154,  37, 127])

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ": This isn't an action movie!!! It uses an occasional voice-over narration (by Ms. Judd) while writi"

Next Char Predictions: 
 "VxFö¦æäF\x85T1|jìw8jq®sbï…ìP!í½\x97Ha\x9eHewj7\\rF\x8d;é5\x8eXÅ\x80`V\x84QV\x10nióå'dbÊL\x9e»06%î,’E\x84Õ\xa0–kåãüww}/uUïD³èn\x85!\x8dûU\\øAÉ"


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 169)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.12901


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS = 10

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
3072/3072 [==============================] - 181s 59ms/step - loss: 1.5391
Epoch 2/10
3072/3072 [==============================] - 186s 61ms/step - loss: 1.2810
Epoch 3/10
3072/3072 [==============================] - 186s 60ms/step - loss: 1.2453
Epoch 4/10
3072/3072 [==============================] - 186s 60ms/step - loss: 1.2289
Epoch 5/10
3072/3072 [==============================] - 186s 60ms/step - loss: 1.2209
Epoch 6/10
3072/3072 [==============================] - 186s 60ms/step - loss: 1.2176
Epoch 7/10
3072/3072 [==============================] - 186s 60ms/step - loss: 1.2169
Epoch 8/10
3072/3072 [==============================] - 186s 60ms/step - loss: 1.2186
Epoch 9/10
3072/3072 [==============================] - 186s 61ms/step - loss: 1.2221
Epoch 10/10
3072/3072 [==============================] - 187s 61ms/step - loss: 1.2278


In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            43264     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 169)            173225    
Total params: 4,154,793
Trainable params: 4,154,793
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"this movie"))

this movie. Ge meets Who was obviously this full child. Shot others, or grase her, he's working on complete. Basic killer can be keated on his apa...d this costumes and insult the Little che main character played in the place knows! Ihagination is better as a plot hobombacker they prasaibe in the memories or so though, or are boring, there's no enjoyment at the impressor /><br />All that's overall, LANGERS are important and myself and mainly plus fash on screen (Dooley), death in film that Billy Steerea, played 'Rube. The musician (N'T be out in the music of many parents of Tom Shedik, in hisly washing through the cast, bunch of suspense in the heart of an objective of "who are playing people's writer from the me! All I can buy ork' however had some surronous scenes. It's not the initial project are badly grabs  ounories and use mysterious humour and chaft of 'The Div. He's too payoffort was that corpier. The only thing worse how he did six Sean's reminiscent of finting in tenant of th

In [33]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [34]:
model1 = tf.keras.Sequential()
model1.add(hub_layer)
model1.add(tf.keras.layers.Dense(16, activation='relu'))
model1.add(tf.keras.layers.Dense(1))

model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense_2 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [35]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [36]:
history = model1.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 3s 87ms/step - loss: 0.6695 - accuracy: 0.5283 - val_loss: 0.6092 - val_accuracy: 0.5897
Epoch 2/10
30/30 [==============================] - 3s 85ms/step - loss: 0.5461 - accuracy: 0.6873 - val_loss: 0.5067 - val_accuracy: 0.7307
Epoch 3/10
30/30 [==============================] - 2s 83ms/step - loss: 0.4200 - accuracy: 0.8117 - val_loss: 0.4110 - val_accuracy: 0.8214
Epoch 4/10
30/30 [==============================] - 2s 83ms/step - loss: 0.3078 - accuracy: 0.8801 - val_loss: 0.3499 - val_accuracy: 0.8425
Epoch 5/10
30/30 [==============================] - 3s 84ms/step - loss: 0.2267 - accuracy: 0.9184 - val_loss: 0.3193 - val_accuracy: 0.8619
Epoch 6/10
30/30 [==============================] - 3s 86ms/step - loss: 0.1672 - accuracy: 0.9477 - val_loss: 0.3064 - val_accuracy: 0.8645
Epoch 7/10
30/30 [==============================] - 3s 85ms/step - loss: 0.1215 - accuracy: 0.9645 - val_loss: 0.3056 - val_accuracy: 0.8650
Epoch 8/10
30

In [37]:
results = model1.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model1.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3592 - accuracy: 0.8516
loss: 0.359
accuracy: 0.852


In [38]:
test_examples_batch, test_labels_batch = next(iter(test_data.batch(2)))
train_examples_batch1,train_labels_batch1 = next(iter(train_data.batch(10)))
for x in test_examples_batch:
  print(x)

print(test_examples_batch)
x_tensor = tf.constant([generate_text(model, start_string=u"This was ")])
print(x_tensor)


tf.Tensor(b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.", shape=(), dtype=string)
tf.Tensor(b"A blackly 

In [39]:
print(train_examples_batch1)

print(x_tensor)
model1.predict(np.array(x_tensor))

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

array([[-1.1922584]], dtype=float32)